In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import time
import datetime

Generation of Origin and Carrier based features (chosen according to the plots from DataExploration1.ipynb and DataExploration2.ipynb)


In [0]:
filenames = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

big_df = pd.DataFrame()

for name in filenames:
  path = "/content/drive/My Drive/PROJECT 1001/Data3/new_data_" + name + ".csv"
  df = pd.read_csv(path)
  df = df[['OP_UNIQUE_CARRIER', 'ORIGIN', 'TAXI_OUT', 'CARRIER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY']]
  big_df = big_df.append(df)

In [0]:
custom_df = big_df.groupby('OP_UNIQUE_CARRIER').mean().reset_index(level=0).drop('LATE_AIRCRAFT_DELAY', axis=1)
custom_df.to_csv('carrier_lookup.csv')
!cp 'carrier_lookup.csv' 'drive/My Drive/PROJECT 1001/'

In [0]:
custom_df = big_df.groupby('ORIGIN').mean().reset_index(level=0)
custom_df.to_csv('origin_lookup.csv')
!cp 'origin_lookup.csv' 'drive/My Drive/PROJECT 1001/'

Generation of scheduled departure time based features (chosen according to the plots from DataExploration4.ipynb)

In [0]:
filenames = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
timelist = ['00:00:00', '03:00:00', '06:00:00', '09:00:00', '12:00:00', '15:00:00', '18:00:00', '21:00:00', '23:59:00']

In [0]:
big_df = pd.DataFrame()

for name in filenames:
  path = "/content/drive/My Drive/PROJECT 1001/Data3/new_data_" + name + ".csv"
  df = pd.read_csv(path)
  df = df[np.isfinite(df['DEP_DELAY'])]
  df = df[np.isfinite(df['ARR_DELAY'])]
  df['SUB_DEP_DELAY'] = pd.to_datetime(df.DEP_DELAY, unit='m').dt.strftime('%H:%M')
  df['DEP_TIME'] = df['DEP_TIME'].astype('int')
  df['DEP_TIME'] = df['DEP_TIME'].astype('str')
  df['DEP_TIME'] = df['DEP_TIME'].replace('2400', '0000')
  df['DEP_TIME'] = df['DEP_TIME'].str.pad(width=4, side='left', fillchar='0')
  df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H%M', errors='coerce').dt.strftime('%H:%M')
  # Conversion to datetime
  df['SUB_DEP_DELAY'] = df['SUB_DEP_DELAY'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M'))
  df['DEP_TIME'] = df['DEP_TIME'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M'))
  # Calculation of scheduled departure time
  df['SCHED_DEP_TIME'] = df['DEP_TIME'] - df['SUB_DEP_DELAY']
  df['SCHED_DEP_TIME'] = df['SCHED_DEP_TIME'] - pd.to_timedelta(df['SCHED_DEP_TIME'].dt.days, unit='d')
  df = df[['SCHED_DEP_TIME', 'ARR_DEL15', 'ARR_DELAY', 'DEP_DEL15', 'DEP_DELAY']]
  big_df = big_df.append(df)

In [0]:
big_df.tail()

,SCHED_DEP_TIME,ARR_DEL15,ARR_DELAY,DEP_DEL15,DEP_DELAY
330111,18:50:00,0.0,-19.0,0.0,-1.0
330112,21:25:00,0.0,-17.0,0.0,-6.0
330113,12:48:00,0.0,11.0,1.0,23.0
330114,23:36:00,1.0,30.0,1.0,37.0
330115,09:59:00,0.0,14.0,0.0,-1.0


In [0]:
# Creation of time bins for taking means of delay related attributes
timelist = ['00:00:00', '03:00:00', '06:00:00', '09:00:00', '12:00:00', '15:00:00', '18:00:00', '21:00:00', '24:00:00']
bins = []
for i in range(len(timelist)):
  bins.append(pd.Timedelta(timelist[i]))
bins

[Timedelta('0 days 00:00:00'),
 Timedelta('0 days 03:00:00'),
 Timedelta('0 days 06:00:00'),
 Timedelta('0 days 09:00:00'),
 Timedelta('0 days 12:00:00'),
 Timedelta('0 days 15:00:00'),
 Timedelta('0 days 18:00:00'),
 Timedelta('0 days 21:00:00'),
 Timedelta('1 days 00:00:00')]

In [0]:
big_df['TIME_GROUP'] = pd.cut(big_df['SCHED_DEP_TIME'], bins, labels=timelist[:8])
big_df['TIME_GROUP'].tail()

330111    18:00:00
330112    21:00:00
330113    12:00:00
330114    21:00:00
330115    09:00:00
Name: TIME_GROUP, dtype: category
Categories (8, object): [00:00:00 < 03:00:00 < 06:00:00 < 09:00:00 < 12:00:00 < 15:00:00 < 18:00:00 <
                         21:00:00]

In [0]:
custom_df = big_df.groupby('TIME_GROUP').mean().reset_index(level=0)
custom_df.to_csv('time_group_lookup.csv')
!cp 'time_group_lookup.csv' 'drive/My Drive/PROJECT 1001/'